INSTALL PYTUBE

In [1]:

## Install pyTube3 https://github.com/nficano/pytube
!pip install git+https://github.com/pytube/pytube


^C


In [ ]:

#importing the module 
from pathlib import Path
from pytube import YouTube
video2Index = YouTube('https://www.youtube.com/watch?v=ijtKxXiS4hE').streams.first().download()
video_name = Path(video2Index).stem



CREATE AN AZURE VIDEO INDEXER INSTANCE

In [ ]:
pip install video-indexer

INITIALIZE CLIENT

In [ ]:
from video_indexer import VideoIndexer

vi = VideoIndexer(vi_subscription_key='5e7d57663db14ee58a06385263836438',
                  vi_location='trial',
                  vi_account_id='b8f7e815-9386-40e3-a527-2a6743630cef')

UPLOAD VIDEO

In [ ]:
video_id = vi.upload_to_video_indexer(
              input_filename = video2Index,
              video_name=video_name, #must be unique
              video_language='English')

GET VIDEO INFO

In [ ]:
info = vi.get_video_info(video_id, video_language='English')


EXTRACT KEYFRAMES ID

In [ ]:
keyframes = []
for shot in info["videos"][0]["insights"]["shots"]:
    for keyframe in shot["keyFrames"]:
        keyframes.append(keyframe["instances"][0]['thumbnailId'])

GET KEYFRAMES ID

In [ ]:
for keyframe in keyframes:
    img_str = vi.get_thumbnail_from_video_indexer(video_id,    
                                                  keyframe)
img_str

Use the Azure Computer Vision API to Extract Popular Brands from Key Frames

INSTALL COMPUTERVISION CLIENT

In [ ]:
pip install --upgrade azure-cognitiveservices-vision-computervision

In [ ]:
pip install pillow

AUTHRNTICATE THE CLIENT

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time



GET VARIABLES FOR ENDPOINT AND APIKEYS

In [ ]:
subscription_key_CV = "c31039d57019480baf596f1ae6acdfa8"
endpoint_CV = "https://cognitivegeneral.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint_CV, CognitiveServicesCredentials(subscription_key_CV))

VISUALIZE KEYFRAMES

In [ ]:
import time
from IPython.display import display
from PIL import Image
import io

timeout_interval, timeout_time = 5, 10.0
image_features = ["brands"]
for index, keyframe in enumerate(keyframes):
    if index % timeout_interval == 0:
        print("Trying to prevent exceeding request limit waiting {}  seconds".format(timeout_time))
        time.sleep(timeout_time)
# Get KeyFrame Image Byte String From Video Indexer
img_str = vi.get_thumbnail_from_video_indexer(video_id, keyframe)
# Convert Byte Stream to Image Stream
img_stream = io.BytesIO(img_str)
# Analyze with Azure Computer Vision
cv_results = computervision_client.analyze_image_in_stream(img_stream, image_features)
print("Detecting brands in keyframe {}: ".format(keyframe))
if len(cv_results.brands) == 0:
    print("No brands detected.")
else:
    for brand in cv_results.brands:
        print("'{}' brand detected with confidence {:.1f}% at location {}, {}, {}, {}".format( brand.name, brand.confidence * 100, brand.rectangle.x, brand.rectangle.x + brand.rectangle.w, brand.rectangle.y, brand.rectangle.y + brand.rectangle.h))

 Key Frames to Azure Custom Vision API to Custom Detect Brands¶


In [ ]:
!pip install azure-cognitiveservices-vision-customvision


INITIALIZE CUSTOM VISION CLIENT

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
prediction_threshold = .8
prediction_key =  "ee27e51576d049fdb0d180bb91f1fa89"
ENDPOINT = "https://cvtestayo-prediction.cognitiveservices.azure.com/"
project_id = "a6161fd5-c077-45f6-987f-0b486aa07557"
published_name = "Iteration1"
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

PREDICT KEYFRAMES WITH CUSTOM VISION MODEL

In [ ]:

import time
timeout_interval, timeout_time = 5, 10.0
image_features = ["brands"]

for index, keyframe in enumerate(keyframes):
    if index % timeout_interval == 0:
        print("Trying to prevent exceeding request limit waiting {} seconds".format(timeout_time))
        time.sleep(timeout_time)
    # Get KeyFrame Image Byte String From Video Indexer
    img_str = vi.get_thumbnail_from_video_indexer(video_id, keyframe)
    # Convert Byte Stream to Image Stream
    img_stream = io.BytesIO(img_str)  
    # Analyze with Azure Computer Vision
    cv_results = predictor.detect_image(project_id, published_name, img_stream)
    predictions = [pred for pred in cv_results.predictions if pred.probability > prediction_threshold]
    print("Detecting brands in keyframe {}: ".format(keyframe))
    if len(predictions) == 0:
        print("No custom brands detected.")
    else:
        for brand in predictions:
            print("'{}' brand detected with confidence {:.1f}% at location {}, {}, {}, {}".format( \
                brand.tag_name, brand.probability * 100, brand.bounding_box.left, brand.bounding_box.top, \
                brand.bounding_box.width, brand.bounding_box.height))